In [66]:
import numpy as np 
from functools import partial 
from concurrent.futures import ThreadPoolExecutor 
from queue import Queue
from threading import Thread
from threading import Lock


### Homework 07: Concurrency

## Due Date: Apr 5, 2023, 11:59pm

#### Firstname Lastname: Buz Galbraith

#### E-mail: wbg231@nyu.edu

#### Enter your solutions and submit this notebook


---

**Problem 1** **(60 Points)**

Let us consider the Gamma function, or the Euler integral of the second kind: 

$$\Gamma(x) = \int_{0} ^ \infty t ^{x - 1} e^{-t} dt, $$

and in this HW we consider real $x > 0$.

(Here is more on the Gamma function https://en.wikipedia.org/wiki/Gamma_function .
It is not needed for this HW assignment.) 

**1.1 (Points 15)**: 

Write a function (in the cell below) that sequentially calculates the given Gamma integral.


In [133]:
def calculate_gamma(x, bound_1, bound_2, number_of_steps):
    # sequential version to calculate Gamma(x):
    # where we approximate the given integral,
    # like this a discrete sum in number_of_steps
    # equidistant points on the interval [bound_1, bound_2]
    # return Gamma(x)
    dt=((bound_2-bound_1)/number_of_steps)
    t=np.linspace(bound_1, bound_2, number_of_steps)
    term_1=np.exp(-1*t)
    term_2=np.multiply.reduce([t] * (x-1))
    dt=t[1]-t[0]
    print(dt)
    out=np.sum(term_1*term_2)*dt
    return out



**1.2 (Points 5)** 

Evaluate, $\Gamma(6)$ by using `calculate_gamma(x, bound_1, bound_2, number_of_steps)` and the error of this computation.


As arguments, use `x=6, bound_1=0, bound_2=1000, number_of_steps=10_000_000`. We know that $\Gamma(x) = x!$, so $\Gamma(6) = 5! = 120$. 


In [134]:
x=6
bound_1=0 
bound_2=1000
number_of_steps=10_000_000

arpoxmiation=calculate_gamma(x, bound_1, bound_2, number_of_steps)
error=120-arpoxmiation
print("My function aproximates the given function call as {0}\n\
this results in an error of {1}".format(arpoxmiation, error))


0.000100000010000001
My function aproximates the given function call as 120.00000000000006
this results in an error of -5.684341886080802e-14


---

Write two functions to calculate $\Gamma(x)$ by using:



**1.3.1 (Points 15)**
**threading** with N=4 threads; 



    

In [139]:
chunks = [(i, i + 100) for i in range(bound_1, bound_2, 100)] ## defines chunks to work on. 
def calculate_gamma_range(x,t):    
    dt=t[1]-t[0]
    term_1=np.exp(-1*t)
    term_2=np.multiply.reduce([t] * (x-1))
    return np.sum(term_1*term_2)*dt
chunks = [(i, i + 100) for i in range(bound_1, bound_2, 100)]
number_of_steps=10_000_000
n_chunk_steps=number_of_steps//len(chunks)
bound_1_chunk, bound_2_chunk=chunks[0]
t_chunk=np.linspace(bound_1_chunk, bound_2_chunk, n_chunk_steps)
calculate_gamma_range(x,t_chunk)

120.00000000000003

In [141]:
y=0
def calc_gamma_chunk(q,x,n_chunk_steps):
    while True:
        bound_1_chunk, bound_2_chunk = q.get()
        t_chunk=np.linspace(bound_1_chunk, bound_2_chunk, n_chunk_steps)
        global y
        with lock:  # force synchronization
            y = y+calculate_gamma_range(x,t_chunk)
            q.task_done()
def multi_thread_gamma(x,bound_1, bound_2,number_of_steps,num_threads):
    chunks = [(i, i + 100) for i in range(bound_1, bound_2, 100)] ## defines so we can work across smaller ranges of length 100. 
    lock = Lock()
    q = Queue()
    for chunk in chunks:
        q.put(chunk)
    n_chunk_steps=number_of_steps//len(chunks)
    for i in range(num_threads):
        worker = Thread(target=calc_gamma_chunk, args=(q,x,n_chunk_steps))
        worker.setDaemon(True) # this stop the threads when the program quits  
        worker.start()         # start the threads
    q.join()
multi_thread_gamma(x,bound_1, bound_2,number_of_steps,4)
y

/tmp/ipykernel_1024/187809148.py:19: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  worker.setDaemon(True) # this stop the threads when the program quits


120.00000000000003

**1.3.2 (Points 15)**
**multiprocessing** with N=4 processes. 



**1.3.3 (Points 10)** 
Compare the times of the three versions and write a short explanation of what you are observing.


How does the answer change when N=8 and why?

---

**Problem 2 (40 points)**

__Website uptime__ is the time that a website or web service is available to the users over a given period.

The task is to build an application that checks the uptime of websites. 

- The application should go over a list of website URLs and checks if those websites are up.
- Instead of performing a classic HTTP GET request, it performs a HEAD request so that it does not affect traffic significantly.
- If the HTTP status is in the danger ranges (400+, 500+), a message is casted. 

Here are some useful functions:

In [ ]:
#### _website uptimer_ ####

import time
import logging
import requests
 
class WebsiteDownException(Exception):
    pass
 
def ping_website(address, timeout=20):
    """
    Check if a website is down. A website is considered down 
    if either the status_code >= 400 or if the timeout expires
     
    Throw a WebsiteDownException if any of the website down conditions are met
    """
    try:
        response = requests.head(address, timeout=timeout)
        if response.status_code >= 400:
            logging.warning("Website %s returned status_code=%s" % (address, response.status_code))
            raise WebsiteDownException()
    except requests.exceptions.RequestException:
        logging.warning("Timeout expired for website %s" % address)
        raise WebsiteDownException()
         
def check_website(address):
    """
    Utility function: check if a website is down, if so, notify the user
    """
    try:
        ping_website(address)
    except WebsiteDownException:
        print('The websie ' + address + ' is down')

---

You need a website list to try our system out. Create your own list or use the following one. 

---

In [ ]:
WEBSITE_LIST = [
    'http://amazon.co.uk',
    'http://amazon.com',
    'http://facebook.com',
    'http://google.com',
    'http://google.fr',
    'http://google.es',
    'http://google.co.uk',
    'http://gmail.com',
    'http://stackoverflow.com',
    'http://github.com',
    'http://heroku.com',
    'http://really-cool-available-domain.com',
    'http://djangoproject.com',
    'http://rubyonrails.org',
    'http://basecamp.com',
    'http://trello.com',
    'http://shopify.com',
    'http://another-really-interesting-domain.co',
    'http://airbnb.com',
    'http://instagram.com',
    'http://snapchat.com',
    'http://youtube.com',
    'http://baidu.com',
    'http://yahoo.com',
    'http://live.com',
    'http://linkedin.com',
    'http://netflix.com',
    'http://wordpress.com',
    'http://bing.com',
]

---

A serial version of the _website uptimer_ can be written as: 

---


In [ ]:
import time
 
start_time = time.time()
 
for address in WEBSITE_LIST:
    check_website(address)
         
end_time = time.time()        
 
print("Time for Serial: %ssecs" % (end_time - start_time))

You should build two versions of the **website uptimer**, by using:

**2.1 (Points 15)**
**threading** with N=4 threads; 

**2.2 (Points 15)**
**multiprocessing** with N=4 processes. 


**2.3 (Points 10)** 

Compare the times of the three versions and write a short explanation of what you are observing.

How does the answer change when N=8 and why?
